In [2]:
import numpy as np
import pandas as pd

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xAuU1omCJnbdpmZrmniLARNtEFkvCBi4AogjiRajPwFZ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'churnmodellingdeployment-donotdelete-pr-o1jnaufdavzxjf'
object_key = 'Churn_Modelling.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.head()



,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
dataset.isnull().any()

RowNumber          False
CustomerId         False
Surname            False
CreditScore        False
Geography          False
Gender             False
Age                False
Tenure             False
Balance            False
NumOfProducts      False
HasCrCard          False
IsActiveMember     False
EstimatedSalary    False
Exited             False
dtype: bool

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
dataset["Geography"] = le.fit_transform(dataset["Geography"])

In [6]:
type(dataset)

pandas.core.frame.DataFrame

In [7]:
dataset['Gender'] = le.fit_transform(dataset['Gender'])

In [8]:
x = dataset.iloc[:,3:13].values
y = dataset.iloc[:,13].values

In [9]:
dataset

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,2,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,1,1,42,3,75075.31,2,1,0,92888.52,1


In [10]:
x


array([[6.1900000e+02, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [6.0800000e+02, 2.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [5.0200000e+02, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [7.0900000e+02, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [7.7200000e+02, 1.0000000e+00, 1.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [7.9200000e+02, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [11]:
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
z = one.fit_transform(x[:,1:2]).toarray()
x = np.delete(x,1,axis = 1)
x = np.concatenate((z,x),axis = 1)

In [12]:
z


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [13]:
x

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)

In [15]:
x.shape

(10000, 12)

In [16]:
x_train.shape

(8000, 12)

In [17]:
x_test.shape

(2000, 12)

In [18]:
from sklearn.model_selection import train_test_split
x1 = [1,2,3,4,5,6,7,8,9,10]
y1 = [1,0,1,0,1,0,1,0,1,0]
for i in range(5):
    x_train1,x_test1,y_train1,y_test1 = train_test_split(x1,y1,test_size = 0.2,random_state = 2)
    print(x_train1 , "with random state")

[6, 1, 8, 3, 4, 7, 10, 9] with random state
[6, 1, 8, 3, 4, 7, 10, 9] with random state
[6, 1, 8, 3, 4, 7, 10, 9] with random state
[6, 1, 8, 3, 4, 7, 10, 9] with random state
[6, 1, 8, 3, 4, 7, 10, 9] with random state


In [19]:
for i in range(5):
    x_train1,x_test1,y_train1,y_test1 = train_test_split(x1,y1,test_size = 0.2)
    print(x_train1, "without random state")

[7, 4, 2, 3, 6, 1, 9, 5] without random state
[10, 1, 4, 2, 8, 7, 5, 9] without random state
[10, 9, 3, 7, 6, 5, 2, 1] without random state
[1, 3, 7, 10, 6, 4, 2, 5] without random state
[3, 10, 5, 2, 9, 1, 6, 8] without random state


In [20]:
from sklearn.ensemble import RandomForestClassifier
forest_reg = RandomForestClassifier(n_estimators=10, criterion='entropy',random_state=42)
forest_reg.fit(x_train, y_train)


RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

from sklearn.ensemble import RandomForestClassifier
forest_reg = RandomForestClassifier(n_estimators=10, criterion='entropy',random_state=42)
forest_reg.fit(x_train, y_train)

In [21]:
x_train[0]

array([1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 6.8600000e+02,
       1.0000000e+00, 3.2000000e+01, 6.0000000e+00, 0.0000000e+00,
       2.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.7909326e+05])

In [22]:
!pip install ibm_watson_machine_learning

In [23]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com" ,
                    "apikey":"vmuginDGnrEtQBTC2Ddw0ieeigysiFG2OLwSGG4TsE7-"
                  }
client = APIClient(wml_credentials)


In [24]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()  
    #print(space)
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name) ['metadata']['id'])

In [25]:
space_uid = guid_from_space_name(client, 'model_deployment')
print("Space UID = " + space_uid)

Space UID = 893a0360-a3f6-4fe5-ba12-450754997b2e


In [26]:
client.set.default_space(space_uid)

'SUCCESS'

In [27]:
!pip install scikit-learn

In [28]:
client.software_specifications.list()

----------------------------  ------------------------------------  ----  ------------  --------------------------
NAME                          ID                                    TYPE  STATE         REPLACEMENT
default_py3.6                 0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base  retired       runtime-22.2-py3.10
autoai-ts_rt23.1-py3.10       01ce9391-1a79-5a33-94fb-2e134337f314  base  supported
kernel-spark3.2-scala2.12     020d69ce-7ac1-5e68-ac1a-31189867356a  base  retired
pytorch-onnx_1.3-py3.7-edt    069ea134-3346-5748-b513-49120e15d288  base  retired
tensorflow_rt23.1-py3.10      079a91e0-245f-5269-8926-3c20b28f37dc  base  supported
scikit-learn_0.20-py3.6       09c5a1d0-9c1e-4473-a344-eb7b665ff687  base  retired       runtime-22.2-py3.10
spark-mllib_3.0-scala_2.12    09f4cff0-90a7-5899-b9ed-1ef348aebdee  base  retired
pytorch-onnx_rt22.1-py3.9     0b848dd4-e681-5599-be41-b5f6fccc6471  base  retired       pytorch-onnx_rt22.2-py3.10
ai-function_0.1-py3.6         0cdb0f1e-5

,NAME,ID,TYPE,STATE,REPLACEMENT
0,default_py3.6,0062b8c9-8b7d-44a0-a9b9-46c416adcbd9,base,retired,runtime-22.2-py3.10
1,autoai-ts_rt23.1-py3.10,01ce9391-1a79-5a33-94fb-2e134337f314,base,supported,
2,kernel-spark3.2-scala2.12,020d69ce-7ac1-5e68-ac1a-31189867356a,base,retired,
3,pytorch-onnx_1.3-py3.7-edt,069ea134-3346-5748-b513-49120e15d288,base,retired,
4,tensorflow_rt23.1-py3.10,079a91e0-245f-5269-8926-3c20b28f37dc,base,supported,
5,scikit-learn_0.20-py3.6,09c5a1d0-9c1e-4473-a344-eb7b665ff687,base,retired,runtime-22.2-py3.10
6,spark-mllib_3.0-scala_2.12,09f4cff0-90a7-5899-b9ed-1ef348aebdee,base,retired,
7,pytorch-onnx_rt22.1-py3.9,0b848dd4-e681-5599-be41-b5f6fccc6471,base,retired,pytorch-onnx_rt22.2-py3.10
8,ai-function_0.1-py3.6,0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda,base,retired,runtime-22.2-py3.10
9,shiny-r3.6,0e6e79df-875e-4f24-8ae9-62dcc2148306,base,retired,


In [29]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.2-py3.10")
software_spec_uid

'b56101f1-309d-549b-a849-eaa63f77b2fb'

In [ ]:
try:
    model_details = client.repository.store_model(model=forest_reg,meta_props={
        client.repository.ModelMetaNames.NAME:"Churn_Modelling",
        client.repository.ModelMetaNames.TYPE:"runtime-22.2-py3.10",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid})
model_id = client.repository.get_model_uid(model_details)
catch(err):
    print(err)

In [ ]:
model_id